In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI, P_PHONE
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np

In [2]:
request_url = f"http://api.weatherapi.com/v1/forecast.json?key={API_KEY_WAPI}&q=mexico city&days=1&aqi=no&alerts=no"
response = requests.get(request_url).json()

## Creando dataframes

In [3]:
def get_forecast(response, i):
    fecha = response["forecast"]["forecastday"][0]["hour"][i]['time'].split()[0]
    hora = int(response["forecast"]["forecastday"][0]["hour"][i]['time'].split()[1].split(':')[0])
    condicion = response["forecast"]["forecastday"][0]["hour"][i]['condition']['text']
    temperatura = response["forecast"]["forecastday"][0]["hour"][i]['temp_c']
    rain = response["forecast"]["forecastday"][0]["hour"][i]['will_it_rain']
    prob_rain = response["forecast"]["forecastday"][0]["hour"][i]['chance_of_rain']
    
    return fecha, hora, condicion, temperatura, rain, prob_rain


    

In [4]:
datos = []

for i in tqdm(range( len(response["forecast"]["forecastday"][0]["hour"])), colour ="MAGENTA"):
    datos.append(get_forecast(response, i))
    

100%|███████████████████████████████████████| 24/24 [00:00<00:00, 212818.81it/s]


In [5]:
datos

[('2023-11-28', 0, 'Overcast', 13.1, 0, 0),
 ('2023-11-28', 1, 'Overcast', 15.0, 0, 0),
 ('2023-11-28', 2, 'Overcast', 12.4, 0, 0),
 ('2023-11-28', 3, 'Overcast', 12.3, 0, 0),
 ('2023-11-28', 4, 'Overcast', 12.2, 0, 0),
 ('2023-11-28', 5, 'Overcast', 12.1, 0, 0),
 ('2023-11-28', 6, 'Cloudy', 12.0, 0, 0),
 ('2023-11-28', 7, 'Overcast', 12.0, 0, 0),
 ('2023-11-28', 8, 'Overcast', 12.0, 0, 0),
 ('2023-11-28', 9, 'Overcast', 12.1, 0, 0),
 ('2023-11-28', 10, 'Overcast', 12.2, 0, 0),
 ('2023-11-28', 11, 'Overcast', 12.9, 0, 0),
 ('2023-11-28', 12, 'Cloudy', 13.7, 0, 0),
 ('2023-11-28', 13, 'Cloudy', 14.6, 0, 0),
 ('2023-11-28', 14, 'Cloudy', 16.0, 0, 0),
 ('2023-11-28', 15, 'Partly cloudy', 17.4, 0, 0),
 ('2023-11-28', 16, 'Cloudy', 17.9, 0, 0),
 ('2023-11-28', 17, 'Cloudy', 17.7, 0, 0),
 ('2023-11-28', 18, 'Overcast', 17.2, 0, 0),
 ('2023-11-28', 19, 'Overcast', 16.6, 0, 0),
 ('2023-11-28', 20, 'Partly cloudy', 15.0, 0, 0),
 ('2023-11-28', 21, 'Partly cloudy', 14.0, 0, 0),
 ('2023-11-28', 2

In [6]:
cols = ['fecha', 'hora', 'condicion', 'temperatura', 'lluvia', 'prob_lluvia']
df = pd.DataFrame(datos,columns = cols)
df

,fecha,hora,condicion,temperatura,lluvia,prob_lluvia
0,2023-11-28,0,Overcast,13.1,0,0
1,2023-11-28,1,Overcast,15.0,0,0
2,2023-11-28,2,Overcast,12.4,0,0
3,2023-11-28,3,Overcast,12.3,0,0
4,2023-11-28,4,Overcast,12.2,0,0
5,2023-11-28,5,Overcast,12.1,0,0
6,2023-11-28,6,Cloudy,12.0,0,0
7,2023-11-28,7,Overcast,12.0,0,0
8,2023-11-28,8,Overcast,12.0,0,0
9,2023-11-28,9,Overcast,12.1,0,0


## Template

In [7]:
msg = '\nEl pronostico del tiempo hoy a las\n {} es de {}'.format(df["hora"][15],df["condicion"][15])
msg

'\nEl pronostico del tiempo hoy a las\n 15 es de Partly cloudy'

# Mensaje msm por twilio

In [8]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid,auth_token)
message = client.messages \
                .create(
                    body = msg,
                    from_=PHONE_NUMBER,
                    to= P_PHONE
                )
print('Mensaje enviado' + message.sid)

Mensaje enviadoSM0b54d875668a56e9c25bb9a51a201add
